## This demo script trains an MMD VAE on a public dataset, then selects representative samples for N iterations.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
###################################
import argparse
import numpy as np
import tensorflow as tf
print('TF v: ', tf.__version__)
import tensorflow_datasets as tfds
import os
import sys
import utils
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

# from learn_representation import train_infoVAE
import learn_representation, utils

/home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/firat/b

TF v:  1.14.0
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



### learn representative space

In [2]:
logdir = os.path.join(os.getcwd(), 'tmp')
# logdir='/home/fozdemir/saved_models/AL_BSQ/norm_ldims5'
batch_size=300
print("Latent dims: 10, lambda_mmd 1e3, batch_size: %d" % (batch_size))
print("Logdir: %s" % (logdir))
##########################################################################
######### Ideally, one better run this outside of the notebook. ##########
# !python learn_representation.py -l {logdir} --ndims_latent 10 --lambda_mmd 1000 --batch_size {batch_size} 
##########################################################################

Latent dims: 10, lambda_mmd 1e3, batch_size: 300
Logdir: /home/firat/projects/al-bsq/tmp


### load function to map onto representative space

In [3]:
# import importlib
# importlib.reload(utils)

sampler_obj = utils.AL_BSQ_Sampler(log_dir=logdir)

2021-01-03 15:03:16,600 From /home/firat/projects/al-bsq/utils.py:167: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

2021-01-03 15:03:16,602 Overwrite dataset info from restored data version.
2021-01-03 15:03:16,607 Reusing dataset mnist (/home/firat/mnist/1.0.0)
2021-01-03 15:03:16,608 Constructing tf.data.Dataset for split train, from /home/firat/mnist/1.0.0


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


2021-01-03 15:03:16,762 From /home/firat/projects/al-bsq/utils.py:179: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


2021-01-03 15:03:16,775 From /home/firat/projects/al-bsq/utils.py:184: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.



2021-01-03 15:03:16,779 From /home/firat/projects/al-bsq/utils.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



...

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-01-03 15:03:17,632 From /home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from /home/firat/projects/al-bsq/tmp/model.ckpt-10000000


2021-01-03 15:03:17,636 Restoring parameters from /home/firat/projects/al-bsq/tmp/model.ckpt-10000000


batch finished


In [4]:
# Sample representative samples
# sampler_obj = utils.AL_BSQ_Sampler(log_dir=log_dir)
num_al_iterations = 1 # sample most representative samples 1 time in a row for this demo.
num_query = 10 # query 10 most representative samples at a time.
inds_an = sampler_obj.inds_an.copy()
l_inds_an = [inds_an]
for i_al in range(num_al_iterations):
    inds_query = sampler_obj.query_samples(inds_an=inds_an, num_query=num_query)
    inds_an = np.concatenate((inds_an, inds_query), axis=0)
    l_inds_an.append(inds_an.copy())
    logging.info('Queried indices for step %d:\n%s' % (i_al+1, str(inds_query)))

2021-01-03 15:03:55,127 Queried indices for step 1:
[10149 35891 47277 18102 27386 15902  9930 32409  7814 21304]


In [5]:
## Alternatively, use commandline script
!python demo.py --logdir {logdir} --ndims_latent 10 --lambda_mmd 1000 --batch_size {batch_size} 

/home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/firat/b

2021-01-03 15:07:51,565 From /home/firat/binaries/miniconda3/envs/tf114cpu/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2021-01-03 15:07:51,567 Restoring parameters from /home/firat/projects/al-bsq/tmp/model.ckpt-10000000
2021-01-03 15:07:51.615978: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.
2021-01-03 15:07:51,719 From /home/firat/projects/al-bsq/learn_rep